# 6. BBox Regression Network

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

from dataset import *
from vnet import *
from training import *
from niiutility import show_image, show_batch_image

%matplotlib inline
%load_ext autoreload
%autoreload 2

## 6.1 Setup Torch Global Variable, load memory map 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, sampler, SubsetRandomSampler
from torchvision import transforms, utils

import torch.nn.functional as F  # useful stateless functions
import torchvision.transforms as T

#------------------------------- GLOBAL VARIABLES -------------------------------------#

USE_GPU = True
BATCH_SIZE = 2
NUM_WORKERS = 6
NUM_TRAIN = 80
LEARNING_RATE = 1e-2

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
    print('using GPU for training')
else:
    device = torch.device('cpu')

using GPU for training


* BvMaskDataset, return image and bbox tuple of 6

In [3]:
#-------------------------LOAD THE DATA SET-------------------------------------------#
regen = False

if regen:

    data_index = np.arange(107)
    data_index = np.delete(data_index, 46)
    data_idnex = np.random.shuffle(data_index)
else:
    data_index = np.array ([50,17,81,39,36,88,33,77,7,1,52,43,34,40,41,18,72,58,51,
                  63,78,35,16,79,0,89,70,67,60,13,76,8,2,47,4,97,29,85,32,
                  55,30,49,44,11,101,22,37,10,92,68,5,64,105,95,20,38,99,
                  84,86,91,96,71,98,104,45,69,103,27,19,59,73,106,93,24,80,
                  66,28,90,3,102,31,26,94,62,54,48,12,61,87,42,65,74,53,57,
                  14,56,83,100,25,6,75,82,23,9,21,15])
    
dataset_trans = BvMaskDataset(data_index, 
                         transform=transforms.Compose([
                             RandomFilp(0.5),
                             RandomAffine(180, 15, 1.2),
                             downSample(2)
                         ])
                     )

#-------------------------CREATE DATA LOADER FOR TRAIN AND VAL------------------------#

data_size = len(dataset_trans)
train_loader = DataLoader(dataset_trans, batch_size=BATCH_SIZE, \
                    sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)),\
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(dataset_trans, batch_size=BATCH_SIZE,
                    sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN,data_size)),\
                    num_workers=NUM_WORKERS)

* Print first 4 batch of data
* 3 order = 122.30930733680725
* 2 order = 91.09534621238708
* 1 order = 69.77253293991089
* 0 order = 62.877673387527466


In [ ]:
start = time.time()
for i_batch, sample_batched in enumerate(train_loader):
    print(i_batch, sample_batched['image'].size(), \
          sample_batched['label'].size())
    show_batch_image(sample_batched['image'],BATCH_SIZE,None)

    # observe 4th batch and stop.
    if i_batch == 3:
        break
end = time.time()
print(end-start)

In [4]:
from vnet import LNet

LoadCKP = False

CKPPath = 'checkpoint2019-04-05 19:46:58.793496.pth'

model = LNet(img_size=(96, 128, 128), out_size=6)
model.apply(weights_init)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=50, verbose=True)
logger = {'train':[], 'validation':[]}

if LoadCKP:
    loadckp(model, optimizer, scheduler, logger, CKPPath, device=device)

In [5]:
#-------------------------NEW MODEL INIT WEIGHT--------------------------------------#

from loss import *

train(model, train_loader, validation_loader, optimizer, scheduler,\
      device=device, dtype=dtype, lossFun=MSE, logger=logger, epochs=5000, startepoch=0)

tensor([[-0.7298,  0.6960, -0.7008,  0.9925, -0.1024, -1.4198],
        [-0.9775,  0.5178, -0.5540,  1.2607,  0.4399, -2.6942]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.7294,  0.5654, -1.7060,  1.6617, -0.3513, -0.7787],
        [-0.3513,  1.1362,  0.7750,  0.4383,  0.3758, -1.4214]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.2132,  0.6925,  0.6533,  1.7403,  0.6038, -0.6992],
        [-0.6640,  2.1095,  0.2848,  3.5039,  0.7753,  0.2882]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[0.8067, 2.1718, 1.4174, 2.8032, 2.2962, 0.2943],
        [0.9946, 3.0748, 1.4138, 2.5076, 1.2758, 1.0482]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([[1.8938, 2.9097, 1.1648, 3.1720, 2.0141, 1.7143],
        [2.1762, 4.0688, 2.5639, 3.6860, 2.1755, 2.2929]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([[1.6426, 4.2055, 1.2775, 3.5374, 2.5030, 2.0919],
        [3.5191, 4.2996, 2.5890, 5.7291, 4.1019, 2.3330]], device='cuda:0',


KeyboardInterrupt: 

In [ ]:
#-------------------------SAVE THE MODEL STATE DICT----------------------------------#
PATH = 'LNET-404.pth'
torch.save(model.state_dict(), PATH)